In [4]:
import pandas as pd
df = pd.read_csv('../data/input/vacsi-a-dep.csv', sep=';')
df

,dep,clage_vacsi,jour,n_dose1
0,01,24,2021-01-15,1
1,01,24,2021-01-16,3
2,01,24,2021-01-20,3
3,01,24,2021-01-21,1
4,01,24,2021-01-22,3
...,...,...,...,...
20297,974,0,2021-01-27,248
20298,976,0,2021-01-25,49
20299,976,0,2021-01-26,34
20300,976,0,2021-01-27,90


In [11]:
df[(df["clage_vacsi"]==0) & (df.dep == "01")].sort_values(by="jour")

,dep,clage_vacsi,jour,n_dose1
18169,01,0,2021-01-06,1
18170,01,0,2021-01-07,65
18171,01,0,2021-01-08,159
18172,01,0,2021-01-09,2
18173,01,0,2021-01-11,134
18174,01,0,2021-01-12,273
18175,01,0,2021-01-13,416
18176,01,0,2021-01-14,465
18177,01,0,2021-01-15,388
18178,01,0,2021-01-16,197
